<a href="https://colab.research.google.com/github/lupis30puc/yelp_bert_random_forest/blob/update-7/RF_BERT_feature_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Set Up

In [1]:
!pip install treeinterpreter
#https://pypi.org/project/treeinterpreter/

In [2]:
!pip install transformers

     |████████████████████████████████| 1.8MB 5.6MB/s 
     |████████████████████████████████| 890kB 18.3MB/s 
     |████████████████████████████████| 2.9MB 24.4MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893261 sha256=020455e93a12332b448be90838f6ec41ce07975296b1d7b61d75b5a12dee473f
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
import string
import math

from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix, accuracy_score, roc_auc_score, roc_curve
from sklearn.ensemble import RandomForestClassifier

import time
import pickle
from treeinterpreter import treeinterpreter as ti
import joblib
from transformers import BertTokenizer
import torch

## Initializing the RF model and the x and y values

In [219]:
# load, no need to initialize the loaded_rf
rf_grid = joblib.load("/content/drive/MyDrive/Yelp/model_128_/rf_grid_not_1000.joblib")

In [221]:
rf_grid.best_params_  # Parameter setting that gave the best results on the hold out data.
rf = rf_grid.best_estimator_ 
print(rf_grid.best_params_)

{'min_samples_leaf': 2, 'n_estimators': 400}


In [8]:
x_train = pd.read_pickle('/content/drive/MyDrive/Yelp/model_128_/binary_train_rf_feb_02')
x_test = pd.read_pickle('/content/drive/MyDrive/Yelp/model_128_/binary_test_rf_feb_02')

In [9]:
y_train = torch.load('/content/drive/MyDrive/Yelp/model_128_/pred_labels_train')
y_test = torch.load('/content/drive/MyDrive/Yelp/model_128_/pred_labels_test')

## Initializing BERT tokenizers and reviews **text**, features and ids

In [40]:
# NO NEED FOR THE DATASET OR YES?
train_df = pd.read_pickle('/content/drive/MyDrive/Yelp/sample_train_10394.pkl')
test_df = pd.read_pickle('/content/drive/MyDrive/Yelp/sample_test_2599.pkl')

train_df.reset_index(drop=True, inplace=True)
test_df.reset_index(drop=True, inplace=True)
#train_df.head()

In [18]:
# launching the saved model tokenizer
tokenizer = BertTokenizer.from_pretrained('/content/drive/MyDrive/Yelp/model_128_/tokenizer_128/')

In [19]:
with open('/content/drive/MyDrive/Yelp/model_128_/feature_names_feb_03.pkl', 'rb') as f:
  feature_names = pickle.load(f)

In [20]:
with open('/content/drive/MyDrive/Yelp/model_128_/unique_ids_feb_03.pkl', 'rb') as f:
  unique_ids = pickle.load(f)

In [21]:
print(len(unique_ids), len(feature_names))

16563 16563


## Feature importances and contributions

In [222]:
importance_rf = rf.feature_importances_

In [237]:
feature_importance = pd.DataFrame({'ids': unique_ids, 'words': feature_names, 'imp': importance_rf})

In [238]:
# sorting the df by importance and reseting the index
# feature_importance[['words', 'imp']].sort_values(by=['imp'], ascending=False)[:15]
feature_importance = feature_importance.sort_values(by=['imp'], ascending=False)
feature_importance.reset_index(drop = True, inplace = True)

In [239]:
# top 15 words
feature_importance[:15] #words with important role 

,ids,words,imp
0,2307,great,0.035311
1,12090,delicious,0.018375
2,6429,amazing,0.016065
3,5409,worst,0.014470
4,5379,friendly,0.012964
5,2293,love,0.012546
6,9202,horrible,0.012401
7,6659,terrible,0.012306
8,2356,asked,0.012191
9,2409,told,0.011806


In [247]:
%%time
#preds_400, bias_400, contributions_400 = ti.predict(random_f2, x_test)
    # takes around 20min to calculate, that's why it is saved!
# loading the preds and contributions
with open('/content/drive/MyDrive/Yelp/random_forest/contributions_400.pkl', 'rb') as f:
  contributions_400 = pickle.load(f)

with open('/content/drive/MyDrive/Yelp/random_forest/pred_400.pkl', 'rb') as f:
  preds_400 = pickle.load(f)

CPU times: user 265 ms, sys: 1.04 s, total: 1.31 s
Wall time: 2.75 s


In [248]:
# number of reviews, and label array | number of reviews, number of features, array for each feature in each review
print(preds_400.shape, contributions_400.shape)

(2599, 2) (2599, 16563, 2)


In [246]:
import pickle
# saving the pred, bias and contributions of RF models
with open('/content/drive/MyDrive/Yelp/random_forest/pred_1000.pkl', 'wb') as f:
  pickle.dump(preds_1000, f)
with open('/content/drive/MyDrive/Yelp/random_forest/bias_1000.pkl', 'wb') as f:
  pickle.dump(bias_1000, f)
with open('/content/drive/MyDrive/Yelp/random_forest/contributions_1000.pkl', 'wb') as f:
  pickle.dump(contributions_1000, f)

with open('/content/drive/MyDrive/Yelp/random_forest/pred_400.pkl', 'wb') as f:
  pickle.dump(preds_400, f)
with open('/content/drive/MyDrive/Yelp/random_forest/bias_400.pkl', 'wb') as f:
  pickle.dump(bias_400, f)
with open('/content/drive/MyDrive/Yelp/random_forest/contributions_400.pkl', 'wb') as f:
  pickle.dump(contributions_400, f)


## Analyzing with contributions

### function for analyzing contributions

In [31]:
import random
def present_sample_review(index, original_df, pred_labels, clean_text=True):
  """
  pred_label is an array
  original df has the columns: label and clean_text/ text

  """
  print("Selected Sample     : %d"%index)
  print("Actual Target Value : %s"%original_df.label[index])
  print("Predicted Value     : %s"%np.argmax(pred_labels[index]))
  if clean_text == True:
    print("" + "\nThe review is: " + "\n" + original_df.clean_text[index])
    print('' + '\nThe review has a length of ' + str(len(original_df.clean_text[index].strip().split(" "))) + ' words.')
  else:
    print("" + "\nThe review is: " + "\n" + original_df.text[index])
    print('' + '\nThe review has a length of ' + str(len(original_df.text[index].strip().split(" "))) + ' words.')

#print("Actual Target Value                      : %s"%y_test[0])
#print("Target Value As Per Treeinterpreter      : %s"%np.argmax(preds_400[0]))


In [176]:
def contributions_analysis(original_df, pred_labels, input_ids, features_df, contributions, print_analysis=True, clean_text=True, index=None):
  """
  contributions is an array
  clean_text=None

  print_analysis can be:
    -True (prints both negative and positive)
    -False (just return dictionaries or lists)
    -negative (prins only negative contributions)
    -positive (prints only positive contributions)

Returns 2 dictionaries with the words as the keys and the contributions as values.
  """
  if index == None:
    review = random.randint(1, len(original_df))
    print("Predictions for random forest 1000 of 2 sample leaves and for a random review")
  else: 
    review = index
    print("Predictions for random forest 1000 of 2 sample leaves and for the review #" + str(review))
  
  present_sample_review(review, original_df, pred_labels, clean_text)

  #extracting only the relevant ids index. the ones that the observed review has
  review_ids = list(features_df[features_df['ids'].isin(input_ids[review])].index) 
  
  print('' + "\nFeature contributions:")
  # creating 2 dictionaries, one for the negative and other for the positive contributions
  negative = {}
  positive = {}
  for i in review_ids:
    c = contributions[review][i] #contributions for the specific id on the observed review
    feature = features_df['words'][i] 
    m = np.argmax(c) #this converts the array numbers into a label
  # if the label is negative:
    if m == 0: 
      negative[feature] = list(c)
    else:
      positive[feature] = list(c)

  def print_negative():
    print(''+ '\nThe ' + str(len(negative)) + ' contributions for negative are: ')
    for key, value in sorted(negative.items(), key=lambda item: item[1], reverse=True): #descending
      print(key, value[1])
  
  def print_positive():
    print(''+ '\nThe ' + str(len(positive)) + ' contributions for positive are: ')
    for key, value in sorted(positive.items(), key=lambda item: item[1]): #ascending
      print(key, value[1])

  if print_analysis == True:
    print_negative()
    print_positive()
  elif print_analysis == 'negative':
    print_negative()
    print(''+ '\nAnd there are ' + str(len(positive)) + ' contributions for positive. ')
  elif print_analysis == 'positive':
    print_positive()
    print(''+ '\nAnd there are ' + str(len(negative)) + ' contributions for negative. ')
  elif print_analysis == False:
    print('There are ' + str(len(negative)) + ' contributions for negative. ')
    print('There are ' + str(len(positive)) + ' contributions for positive. ')
  return negative, positive

In [187]:
def print_negative():
  print(''+ '\nThe ' + str(len(negative)) + ' contributions for negative are: ')
  print(negative)

def print_positive():
  print(''+ '\nThe ' + str(len(positive)) + ' contributions for positive are: ')
  print(positive)

In [188]:
def contributions_analysis2(original_df, pred_labels, input_ids, features_df, contributions, print_analysis=True, clean_text=True, index=None):
  """
  contributions is an array
  clean_text=None

  print_analysis can be:
    -True (prints both negative and positive)
    -False (just return dictionaries or lists)
    -negative (prins only negative contributions)
    -positive (prints only positive contributions)

Returns 2 dictionaries with the words as the keys and the contributions as values.
  """
  if index == None:
    review = random.randint(1, len(original_df))
    print("Predictions for random forest 1000 of 2 sample leaves and for a random review")
  else: 
    review = index
    print("Predictions for random forest 1000 of 2 sample leaves and for the review #" + str(review))
  
  present_sample_review(review, original_df, pred_labels, clean_text)

  #extracting only the relevant ids index. the ones that the observed review has
  review_ids = list(features_df[features_df['ids'].isin(input_ids[review])].index) 
  
  print('' + "\nFeature contributions:")
  # creating a dictionary of the contributions for the specific id on the observed review
  contribs = {}
  for i in review_ids:
    c = contributions[review][i] 
    feature = features_df['words'][i] 
    contribs[feature] = list(c)
  
  # creating a dataframe out of it 
  contrib_df = pd.DataFrame(data=contribs.values(), index = contribs.keys(), columns=["negative", "positive"])
  
  #obtaining the negative and positive contributions
  negative = contrib_df[contrib_df['positive'] <= 0].sort_values(by=['positive']) #ascending positive
  positive = contrib_df[contrib_df['positive'] > 0].sort_values(by=['positive'], ascending=False) #descending positive

  # determining what to print 
  if print_analysis == True:
    print_negative()
    print_positive()
  elif print_analysis == 'negative':
    print_negative()
    print(''+ '\nAnd there are ' + str(len(positive)) + ' contributions for positive. ')
  elif print_analysis == 'positive':
    print_positive()
    print(''+ '\nAnd there are ' + str(len(negative)) + ' contributions for negative. ')
  elif print_analysis == False:
    print('There are ' + str(len(negative)) + ' contributions for negative. ')
    print('There are ' + str(len(positive)) + ' contributions for positive. ')

  # inserting the predicted label at the end
  prediction = contrib_df[["negative", "positive"]].sum()
  contrib_df.loc["PREDICTION"] = np.argmax(prediction)

  return contrib_df

In [167]:
print(contrib_df[contrib_df['positive'] <= 0].sort_values(by=['positive']))

            negative  positive
dirty       0.094578 -0.094578
wouldn      0.055609 -0.055609
sucks       0.054107 -0.054107
driver      0.013504 -0.013504
hotel       0.009854 -0.009854
mention     0.000883 -0.000883
killed      0.000627 -0.000627
drunk       0.000624 -0.000624
[CLS]       0.000000  0.000000
[SEP]       0.000000  0.000000
freak       0.000000  0.000000
PREDICTION  0.000000  0.000000


## running the analysis

In [38]:
with open('/content/drive/MyDrive/Yelp/model_128_/train_ids_128.pkl', 'rb') as f:
    train_ids = pickle.load(f)

with open('/content/drive/MyDrive/Yelp/model_128_/test_ids_128.pkl', 'rb') as d:
    test_ids = pickle.load(d)

In [44]:
test_df.head(2)

,text,label,categories,clean_text
0,Positives: \n- good location \n- staff reasona...,0,"Nail Salons, Beauty & Spas, Day Spas",positives good location staff reasonably frien...
1,"Very clean hotel.jacuzzi is very huge,shower a...",0,"Wedding Planning, Event Planning & Services, B...",clean hotel jacuzzi huge shower bathroom huge ...


In [48]:
test_df['categories'].head() 

0                 Nail Salons, Beauty & Spas, Day Spas
1    Wedding Planning, Event Planning & Services, B...
2                   Home Health Care, Health & Medical
3                                   Food, Coffee & Tea
4    Education, Local Services, Child Care & Day Ca...
Name: categories, dtype: object

In [46]:
# .apply(lambda x : len(x.strip().split(" ")))
len(test_df.clean_text[0].strip().split(" "))

66

In [47]:
len(list(feature_importance_1000[feature_importance_1000['ids'].isin(test_ids[0])].index))

71

In [52]:
len(list(feature_importance_400[feature_importance_400['ids'].isin(test_ids[0])].index))

71

In [70]:
rev0_negative, rev0_positive = contributions_analysis(test_df, preds_400, test_ids, feature_importance_400, contributions_400, print_analysis=False, index=0)

Predictions for random forest 1000 of 2 sample leaves and for the review #0
Selected Sample     : 0
Actual Target Value : 0
Predicted Value     : 0

The review is: 
positives good location staff reasonably friendly negatives dry manis pedis advertise website update site base coat coat blobbed attention coat cover color nail nails filed totally random lenths shapes shorter end fingers longer pedi woman wasn paying attention ended filing skin bled cute tea menu offer definitely expensive getting answer phone pedi station isn comfortable slightly padded bench notes staff looks surprised sure come ask work

The review has a length of 66 words.

Feature contributions:
There are 57 contributions for negative. 
There are 14 contributions for positive. 


In [71]:
rev0_neg, rev0_pos = contributions_analysis(test_df, preds_1000, test_ids, feature_importance_1000, contributions_1000, print_analysis= False, index=0)

Predictions for random forest 1000 of 2 sample leaves and for the review #0
Selected Sample     : 0
Actual Target Value : 0
Predicted Value     : 0

The review is: 
positives good location staff reasonably friendly negatives dry manis pedis advertise website update site base coat coat blobbed attention coat cover color nail nails filed totally random lenths shapes shorter end fingers longer pedi woman wasn paying attention ended filing skin bled cute tea menu offer definitely expensive getting answer phone pedi station isn comfortable slightly padded bench notes staff looks surprised sure come ask work

The review has a length of 66 words.

Feature contributions:
There are 52 contributions for negative. 
There are 19 contributions for positive. 


In [179]:
%%time
n_1639, p_1639 = contributions_analysis(test_df, preds_1000, test_ids, feature_importance_1000, contributions_1000, print_analysis = False, index=1639)

Predictions for random forest 1000 of 2 sample leaves and for the review #1639
Selected Sample     : 1639
Actual Target Value : 0
Predicted Value     : 0

The review is: 
freakin coffee hotel dirty service sucks bar mention coffee wouldn recommend hotel drunk driver killed parents

The review has a length of 16 words.

Feature contributions:
There are 11 contributions for negative. 
There are 7 contributions for positive. 
CPU times: user 4.93 ms, sys: 31 µs, total: 4.96 ms
Wall time: 4.58 ms


In [178]:
%%time
contribs_1639 = contributions_analysis2(test_df, preds_1000, test_ids, feature_importance_1000, contributions_1000, print_analysis= False, index=1639)

Predictions for random forest 1000 of 2 sample leaves and for the review #1639
Selected Sample     : 1639
Actual Target Value : 0
Predicted Value     : 0

The review is: 
freakin coffee hotel dirty service sucks bar mention coffee wouldn recommend hotel drunk driver killed parents

The review has a length of 16 words.

Feature contributions:
There are 11 contributions for negative. 
There are 7 contributions for positive. 
CPU times: user 13.5 ms, sys: 1.02 ms, total: 14.5 ms
Wall time: 17.2 ms


In [180]:
#analyzing a random review
%%time 
_, __ = contributions_analysis(test_df, preds_1000, test_ids, feature_importance_1000, contributions_1000, print_analysis=False)

Predictions for random forest 1000 of 2 sample leaves and for a random review
Selected Sample     : 139
Actual Target Value : 0
Predicted Value     : 1

The review is: 
thought going amazing got perm curls lasting long month curls gone scalp hurting lot perm gotten perms time actually hurt hair cut brought photos helpful recommend place wants japanese style

The review has a length of 30 words.

Feature contributions:
There are 24 contributions for negative. 
There are 8 contributions for positive. 
CPU times: user 9.21 ms, sys: 980 µs, total: 10.2 ms
Wall time: 13.6 ms


In [183]:
%%time
z = contributions_analysis2(test_df, preds_1000, test_ids, feature_importance_1000, contributions_1000, print_analysis=False)

Predictions for random forest 1000 of 2 sample leaves and for a random review
Selected Sample     : 179
Actual Target Value : 1
Predicted Value     : 1

The review is: 
alight nightlife spot good bad arrive hang spend night floating eating talking nightlife routine good female bartender patio little bar area super friendly great good attitude helped girls drink suggestions liked bad female bartender main bar area mix drinks couldn long island iced teas service skills angry life general avoided main bar rounds dismay tips music decent atmosphere cool like good spot try dinner time outdoor patio good furniture hang friends

The review has a length of 71 words.

Feature contributions:
There are 37 contributions for negative. 
There are 24 contributions for positive. 
CPU times: user 12.6 ms, sys: 1.02 ms, total: 13.6 ms
Wall time: 14.3 ms


In [194]:
%%time
z = contributions_analysis2(test_df, preds_1000, test_ids, feature_importance_1000, contributions_1000, print_analysis=False)

Predictions for random forest 1000 of 2 sample leaves and for a random review
Selected Sample     : 670
Actual Target Value : 1
Predicted Value     : 0

The review is: 
tried car came right order wasn ready said going valerie offered drink waited issues

The review has a length of 14 words.

Feature contributions:
There are 12 contributions for negative. 
There are 5 contributions for positive. 
CPU times: user 7.99 ms, sys: 1.88 ms, total: 9.87 ms
Wall time: 12.7 ms


In [193]:
z[z['positive'] <= 0].sort_values(by=['positive'])

,negative,positive
manager,0.048533,-0.048533
bad,0.032448,-0.032448
order,0.015199,-0.015199
arrived,0.013072,-0.013072
saying,0.012301,-0.012301
...,...,...
natural,0.000000,0.000000
club,0.000000,0.000000
[SEP],0.000000,0.000000
shots,0.000000,0.000000


In [186]:
z[z['positive'] > 0].sort_values(by=['positive'], ascending=False)

,negative,positive
PREDICTION,1.000000,1.000000
friendly,-0.047454,0.047454
great,-0.045536,0.045536
spot,-0.019535,0.019535
atmosphere,-0.017749,0.017749
good,-0.009516,0.009516
little,-0.009319,0.009319
outdoor,-0.008517,0.008517
try,-0.006177,0.006177
cool,-0.005466,0.005466


## More about contributions

In [200]:
import random
random_sample = random.randint(1, len(x_test))
print("Selected Sample     : %d"%random_sample)
print("Actual Target Value : %s"%y_test[random_sample])
print("Predicted Value     : %s"%np.argmax(preds_400[random_sample]))

Selected Sample     : 462
Actual Target Value : 1
Predicted Value     : 1


In [240]:
def create_contrbutions_df(contributions, review, features_df, input_ids):
  #extracting only the relevant ids index. the ones that the observed review has
  review_ids = list(features_df[features_df['ids'].isin(input_ids[review])].index) 

  #creating a dictionary of the contributions for the specific id on the observed review
  contribs = {}
  for i in review_ids:
    c = contributions[review][i] 
    feature = features_df['words'][i] 
    contribs[feature] = list(c)
  
  # creating a dataframe out of it 
  contrib_df = pd.DataFrame(data=contribs.values(), index = contribs.keys(), columns=["negative", "positive"])
  # inserting the predicted label at the end
  prediction = contrib_df[["negative", "positive"]].sum()
  contrib_df.loc["PREDICTION"] = np.argmax(prediction)
  return contrib_df


In [241]:
# from: https://coderzcolumn.com/tutorials/machine-learning/treeinterpreter-interpreting-tree-based-models-prediction-of-individual-sample
%%time
contrib_df2 = create_contrbutions_df(contributions_400, random_sample, feature_importance_400, test_ids)
contrib_df2

CPU times: user 8.44 ms, sys: 14 µs, total: 8.45 ms
Wall time: 14 ms


In [242]:
len(contrib_df2[contrib_df['positive'] > 0].sort_values(by=['positive'], ascending=False)) #len positive

38

In [243]:
len(contrib_df2[contrib_df['positive'] <= 0].sort_values(by=['positive'])) #len negative

37

In [211]:
%%time
contri_462 = contributions_analysis2(test_df, preds_400, test_ids, feature_importance_400, contributions_400, print_analysis=False, index=462)

Predictions for random forest 1000 of 2 sample leaves and for the review #462
Selected Sample     : 462
Actual Target Value : 1
Predicted Value     : 1

The review is: 
went sunday dinner kids excited going bobby flay restaurant wanted squid appetizer thought fishy apple blue cheese salad appetizer good small mains fabulous venison tender rich sauce pork tenderloin spicy rub seafood ciappino tipper prawns scallops grouper mussels garlic sweet potato tamale good sides baby carrots mashed potatoes good especially carrots reason stars appetizers good rest good value

The review has a length of 58 words.

Feature contributions:
There are 37 contributions for negative. 
There are 37 contributions for positive. 
CPU times: user 14.2 ms, sys: 42 µs, total: 14.2 ms
Wall time: 19.9 ms


In [244]:
contrib_df2[contrib_df2['positive'] > 0].sort_values(by=['positive'], ascending=False)[1:10]

,negative,positive
good,-0.024474,0.024474
fabulous,-0.015700,0.015700
sweet,-0.015132,0.015132
pork,-0.014186,0.014186
spicy,-0.014072,0.014072
especially,-0.008964,0.008964
tender,-0.008526,0.008526
baby,-0.004453,0.004453
app,-0.004352,0.004352


In [245]:
contrib_df2[contrib_df2['positive'] <= 0].sort_values(by=['positive'])[:10]

,negative,positive
wanted,0.035460,-0.035460
went,0.021359,-0.021359
stars,0.012787,-0.012787
going,0.012405,-0.012405
thought,0.010484,-0.010484
reason,0.006607,-0.006607
rest,0.006501,-0.006501
excited,0.005866,-0.005866
small,0.003396,-0.003396
bobby,0.003379,-0.003379


## Notes:

MAKE A FUNCTION FOR THE THING ABOVE SO IT CAN BE REPLICATED TO WHATEVER REVIEW, EITHER RANDOM OF SPECIFIC AND THAT IT CAN BE SPECIFY IF YOU WANT RESULTS AS DICT OR LIST OR PRINTED ONLY... 

BABY THIS IS IT YOU CAN DO IT!


I need:

predicted labels,
original df to get the clean text / orig text and y set / original labels and len of x set/ original corpus

feature importance df,
input ids list,
contributions array


 Global features and
Specific ones see if they match the reviews